In [22]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import GridSearchCV
from time import time
import joblib


In [23]:
# Chargement des données
DataFrame = pd.read_csv("Data_Arbre.csv")



# Préparation des données
DataFrame.loc[DataFrame['fk_arb_etat'] == 'Essouché', 'fk_arb_etat'] = 1
DataFrame.loc[DataFrame['fk_arb_etat'] != 1, 'fk_arb_etat'] = 0
DataFrame['fk_arb_etat'] = DataFrame['fk_arb_etat'].astype('int')

# Sélectionner les colonnes pertinentes
colonnes_numeriques = ['longitude', 'latitude', 'haut_tot', 'haut_tronc', 'tronc_diam', 'age_estim', 'fk_prec_estim', 'clc_nbr_diag']
colonnes_categorieles = ['clc_quartier', 'clc_secteur', 'fk_stadedev', 'fk_port', 'fk_pied', 'fk_situation', 'fk_revetement', 'fk_nomtech', 'feuillage', 'villeca']
DataFrame.loc[DataFrame['remarquable'] == 'Oui', 'remarquable'] = 1
DataFrame.loc[DataFrame['remarquable'] != 1, 'remarquable'] = 0
DataFrame['remarquable'] = DataFrame['remarquable'].astype('int')


In [27]:
# Encodage des variables catégorielles
X = pd.concat([DataFrame[colonnes_numeriques], pd.get_dummies(DataFrame[colonnes_categorieles]), DataFrame['remarquable']], axis=1)
y = DataFrame['fk_arb_etat']

# Standardisation des données
scaler = StandardScaler().fit(X)
X_scaled = scaler.transform(X)

# Sauvegarder le scaler
# 2 possibilités pour sauvegarder un modèle : joblib et pickle allez voir les doc pour plus d'infos
# joblib est plus rapide et plus efficace pour les gros modèles
# joblib est plus adapté pour les objets sklearn
joblib.dump(scaler, 'scaler.pkl')

# Diviser les données en ensemble d'entraînement et de test
indices = np.arange(X.shape[0])
X_train, X_test, y_train, y_test, indices_train, indices_test = train_test_split(X_scaled, y, indices, test_size=0.3, random_state=42)

data_test_csv = pd.read_csv("Data_Arbre.csv")
data_test = data_test_csv.iloc[indices_test]

data_test.to_json('data_test.json', orient='records')


[[ 0.42896038  1.37374021  1.02314578 ... -0.30669821  0.30669821
  -0.12105172]
 [-1.47150196 -0.34763432 -0.59694528 ... -0.30669821  0.30669821
  -0.12105172]
 [ 0.97441487  0.35896618  0.53711846 ... -0.30669821  0.30669821
  -0.12105172]
 ...
 [ 0.31151847 -1.33289973 -0.43493618 ... -0.30669821  0.30669821
  -0.12105172]
 [ 0.8063264   0.44500884 -1.0829726  ... -0.30669821  0.30669821
  -0.12105172]
 [ 0.64291388 -0.78951003 -0.27292707 ...  3.26053419 -3.26053419
  -0.12105172]]


In [25]:
# Appliquer SMOTE sur l'ensemble d'entraînement uniquement
smote = SMOTE(random_state=42, sampling_strategy=0.2)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

param_grid = {
    #pour savoir quels paramètre mettre dans param_grid, on peut faire une recherche sur Randomforestclassifier sur la doc de sklearn https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html et voir les paramètres qu'on peut mettre
    'n_estimators': [ 70, 80, 90],
    'max_depth': [None, 10, 20, 30, 40],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    #'bootstrap': [True, False],
    #'max_features': ['auto', 'sqrt', 'log2']
}

# Configurer GridSearchCV
# https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html#sklearn.model_selection.GridSearchCV.get_params
# n_jobs = -1 pour utiliser tous les coeurs du processeur
# verbose = 2 pour afficher les logs
# on aurait pu ajouter  refit=True pour entrainer le modèle avec les meilleurs hyperparamètres trouvés
clf = RandomForestClassifier(random_state=42)
grid_search = GridSearchCV(estimator=clf, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1, verbose=2)

# Exécuter GridSearchCV
grid_search.fit(X_train_smote, y_train_smote)

# Obtenir les meilleurs hyperparamètres
best_params = grid_search.best_params_
print("Meilleurs hyperparamètres trouvés : ", best_params)

# Entraîner le modèle avec les meilleurs hyperparamètres

best_clf = grid_search.best_estimator_

# Sauvegarder le meilleur modèle
joblib.dump(best_clf, 'best_random_forest_model.pkl')

# Importance des caractéristiques
importance = best_clf.feature_importances_
indices = np.argsort(importance)[::-1]
print("\nImportance des caractéristiques :")
for f in range(X_train_smote.shape[1]):
    print(f"{X.columns[indices[f]]} : {importance[indices[f]]}")

Fitting 5 folds for each of 135 candidates, totalling 675 fits
Meilleurs hyperparamètres trouvés :  {'max_depth': 30, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 80}

Importance des caractéristiques :
haut_tronc : 0.07548235321308842
fk_prec_estim : 0.06305520022143106
longitude : 0.06080154344943156
clc_nbr_diag : 0.05842746294415226
age_estim : 0.05731531343023211
latitude : 0.054506211299073994
clc_secteur_Parc des Champs-Elysées : 0.04176904131387971
haut_tot : 0.039484129415935806
tronc_diam : 0.03691148116423884
fk_situation_Groupe : 0.03029006576529309
fk_situation_Alignement : 0.027067101954085005
fk_nomtech_SORAUC : 0.025366172555163748
clc_secteur_Cimetière Sud : 0.020294575801554603
fk_pied_gazon : 0.019657012539156617
clc_quartier_Quartier Remicourt : 0.019262967271591123
fk_situation_Isolé : 0.015196520260346708
fk_nomtech_BETPEN : 0.014701996969934645
clc_quartier_Quartier du faubourg d'Isle : 0.013305851509372451
villeca_VILLE : 0.011926144092771233
cl

In [29]:
# Utiliser les caractéristiques les plus importantes pour entraîner un nouveau modèle
X_train_selected = X_train_smote[:, indices[:20]]
best_clf.fit(X_train_selected, y_train_smote)
X_test_selected = X_test[:, indices[:20]]


# Prédiction
y_pred = best_clf.predict(X_test_selected)


# Évaluation du modèle
print("\nAccuracy : ", accuracy_score(y_test, y_pred))
print("\nClassification Report :")
print(classification_report(y_test, y_pred))
print("\nConfusion Matrix :")
print(confusion_matrix(y_test, y_pred, normalize='true'))
print("\naccuracy Score :")
print(accuracy_score(y_test, y_pred))


Accuracy :  0.9802069275753487

Classification Report :
              precision    recall  f1-score   support

           0       0.99      0.99      0.99      2181
           1       0.46      0.26      0.33        42

    accuracy                           0.98      2223
   macro avg       0.72      0.63      0.66      2223
weighted avg       0.98      0.98      0.98      2223


Confusion Matrix :
[[0.99403943 0.00596057]
 [0.73809524 0.26190476]]

accuracy Score :
0.9802069275753487
